In [1]:
# Automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../../FastAi")
#sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
print(sys.path)

['', '/home/sindre/anaconda3/envs/fastai/lib/python36.zip', '/home/sindre/anaconda3/envs/fastai/lib/python3.6', '/home/sindre/anaconda3/envs/fastai/lib/python3.6/lib-dynload', '/home/sindre/anaconda3/envs/fastai/lib/python3.6/site-packages', '/home/sindre/anaconda3/envs/fastai/lib/python3.6/site-packages/IPython/extensions', '/home/sindre/.ipython', '../../FastAi']


In [3]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import os

# Downloading the images

Downloading the images using a program from https://github.com/hardikvasa/google-images-download and running it through bash scripting locally in the notebook. More possible configurations, such as multiple searchwords, different combinations, etc., but this is enough for now. 

To run this method:
> Using pip
> <div class="alert alert-block alert-info">
> \$ pip install google_images_download
</div> <br>
> Manually using CLI
> <div class="alert alert-block alert-info">
> \$ git clone https://github.com/hardikvasa/google-images-download.git <br>
> \$ cd google-images-download && sudo python setup.py install
> </div> <br>  
> Manually using UI
> <div class="alert alert-block alert-info">
> Go to the repo on github ==> Click on *Clone or Download* ==> Click on *Download ZIP* and save it on your local disk. </div>

<div class="alert alert-block alert-warning">
**NOTE** To download > 100 images, please see GitHub 
</div>

### The specified searchwords will be the classes

### Storing the data in such a way that it matches standard PyTorch folder setup
> <div class="alert alert-block alert-info">
*data/train/className* <br />
*data/valid/className*
</div>

### The specified searchwords will be the class names

In [4]:
def download_pictures(searchword, form="jpg", lim=100, directory="data/"):
    if not os.path.isdir(directory):
        !mkdir $directory
    ! googleimagesdownload --keywords $searchword --format $form --limit $lim --output_directory $directory --chromedriver "../../../../Downloads/chromedriver" 
    src_path = directory + searchword
    dst_train_path = directory + "train/"
    dst_valid_path = directory + "valid/"
    if not os.path.isdir(dst_train_path and dst_valid_path):
        ! mkdir $dst_train_path
        ! mkdir $dst_valid_path
    if not os.path.isdir(dst_train_path + searchword and dst_valid_path + searchword):
        train_search_folder = dst_train_path + searchword
        valid_search_folder = dst_valid_path + searchword
        ! mkdir $train_search_folder
        ! mkdir $valid_search_folder
    counter = 0
    number_of_files = len(os.listdir(src_path))
    train_ratio = number_of_files*0.7 
    for file in os.listdir(src_path):
        src = src_path + "/" + file
        dst_end = searchword + "/" + searchword + "." + str(counter) + "." + form
        if(counter <= train_ratio):
            dst = dst_train_path + dst_end
        else:
            dst = dst_valid_path + dst_end
        os.rename(src, dst)
        counter = counter + 1
    delete_dir = directory + searchword
    ! rm -rf $delete_dir

### Specifying the desired classes to classify

In [5]:
class1_name = "Female_face"
class2_name = "Male_face"

In [6]:
%%capture capt
download_pictures(class1_name, lim=1500);
download_pictures(class2_name, lim=1500);

In [7]:
capt.show()


Item no.: 1 --> Item name = Female_face
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1. archetypal-female-_3249633c.jpg
Completed Image ====> 2. 4e7b76f7d680884a05b131a599164cff--unique-faces-model-face.jpg
Completed Image ====> 3. stock-photo-beauty-face-of-the-young-beautiful-woman-isolated-on-white-gorgeous-female-portrait-with-slicked-723534274.jpg
Completed Image ====> 4. close-up-beautiful-female-face-21789058.jpg
Completed Image ====> 5. 9aa0c4d51c6be64845bacbe26a738544.jpg
Completed Image ====> 6. 9354-close-up-female-face.jpg
Completed Image ====> 7. stock-photo-beauty-portrait-of-female-face-with-natural-clean-skin-564737872.jpg
Completed Image ====> 8. f5a0626a80fe6026c0ac65cdc2d8ede2--photography-portraits-photography-people.jpg
Completed Image ====> 9. closeup-beautiful-female-face-isolated-23885901.jpg
Completed Image ====> 10. 300.jpg
Completed Image ====> 11. this-survey-shows-

# Deep Learning
> Utilizing the FastAi framework

In [ ]:
PATH = "data/"
os.listdir(PATH)

In [ ]:
train_path = "train/"
valid_path = "valid/"
print(os.listdir(PATH + train_path))
print(os.listdir(PATH + valid_path))

Possible to get a nicer output of the items in directories? 

In [ ]:
class1_path = class1_name + "/"
class2_path = class2_name + "/"
print("Training folder: ")
print("Class1: " , os.listdir(PATH + train_path + class1_path)[:2])
print("Class2: " , os.listdir(PATH + train_path + class2_path)[:2])

print("Validation folder: ")
print("Class1: " , os.listdir(PATH + valid_path + class1_path)[:2])
print("Class2: " , os.listdir(PATH + valid_path + class2_path)[:2])

In [ ]:
files = os.listdir(f'{PATH}{train_path}{class2_path}')[:1]
files

In [ ]:
img = plt.imread(f'{PATH}{train_path}{class2_path}{files[0]}')
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
img[:4, :4]

## Uncomment if needing to resize images 

> <div class="alert alert-block alert-warning">
Possible problems with it storing them in *tmp* folder
</div>

In [ ]:
#from sklearn import metrics 

#def get_data(sz, bs):
#    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
#    data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)
#    return data if sz > 300 else data.resize(340, 'tmp');

### Setting training values

In [ ]:
arch = resnet34
sz = 299 # One of two standards for ImageNet (224)
bs = 64  # 64 and 128 is standard (pr. Jeremy Howard FastAi)

### Data Augmentation
When training for multiple epochs, the model will start overfitting, which basically means the model is learning to recognizing specific images in the training set, rather than generalizing such that it is relevant for other images than just the ones in the training set. 

One quick fix is to effectively create more data, through data augmentation. This refers to randomly changing the images in ways that shouldn't impact their interpretation, such as horizontal flipping, zooming, and rotating.

In [ ]:
augmentation = transforms_side_on

## Training

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=augmentation, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, ps=0.4)
learn.fit(1e-3, 1)

## Problems because of non-valid image types

> <div class="alert alert-block alert-info">
Can use module *imghdr* to verify image types
</div>

In [ ]:
import imghdr
imghdr.what(f'{PATH}{train_path}{class2_path}{files[0]}')

### Loop through dataset to remove invalid images 

> <div class="alert alert-block alert-info">
But first, interesting to see amount of lost data
</div>

In [ ]:
class1_train = PATH + train_path + class1_path
class2_train = PATH + train_path + class2_path
class1_valid = PATH + valid_path + class1_path
class2_valid = PATH + valid_path + class2_path

In [ ]:
print(class1_name + " training files: " , len(os.listdir(class1_train)))
print(class2_name + " training files: " , len(os.listdir(class2_train)))
print(class1_name + " validation files: " , len(os.listdir(class1_valid)))
print(class2_name + " validation files: " , len(os.listdir(class2_valid)))

In [ ]:
file_paths = [class1_train, 
              class2_train,
              class1_valid, 
              class2_valid,
             ]
for path in file_paths:
    for files in os.listdir(path):
        if imghdr.what(path + files) != 'jpeg':
            #print("Deleting " + files + " in folder " + path + ", should be " + str(imghdr.what(path+files)))
            os.remove(path + files);

In [ ]:
print(class1_name + " training files: " , len(os.listdir(class1_train)))
print(class2_name + " training files: " , len(os.listdir(class2_train)))
print(class1_name + " validation files: " , len(os.listdir(class1_valid)))
print(class2_name + " validation files: " , len(os.listdir(class2_valid)))

### NOTE: Tried to switch image type from *None* to *JPEG* 
> <div class="alert alert-block alert-info">
In order to minimize data loss. 
</div>

## New data set, so new training

In [ ]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, ps=0.6)

In [ ]:
learn.fit(1e-2, 1)

In [ ]:
learn.fit(1e-2, 2, cycle_len=1)

# Locate optimal learning rate

In [ ]:
learn.lr_find()
learn.sched.plot()

## Optimal learning rate is approximately 1e-3

In [ ]:
lr = 1e-3

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

> Unfreezing the first layers, and checking if there is a new optimal learning rate

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.sched.plot()

In [ ]:
lr = ... 

In [ ]:
learn.fit(lr, 3 cycle_len=1, cycle_mult=2)

### Checking acuraccy on validation set using Test Time Augmentation
Using the same transforms method as specified *tfms*

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), 0)

accuracy_np(probs, y)

### Saving the model

In [ ]:
learn.save("Resnet34_binary")

## Visualizing the Data Augmentation

In [ ]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [ ]:
ims = np.stack([get_augs() for i in range(6)])

In [ ]:
plots(ims, rows=2)

# NOTE: Overfitting

### When *validation loss* >> *training* 

> <div class="alert alert-block alert-info">
Data Augmentation, Increase dropout, unfreeze and use differential learning rates
</div>

## Not very good results: 80% accuracy

### Trying to add differential dropout, and longer training 

In [ ]:
#ps=[0.1, 0.3, 0.5]
ps = 0.5

In [ ]:
#learn = ConvLearner.pretrained(arch, data, ps=ps)
#learn.unfreeze()
learn.fit(1e-4, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
lr=np.array([1e-6,1e-5,1e-4])

In [ ]:
#learn.unfreeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), 0)

accuracy_np(probs, y)

In [ ]:
learn.save("Resnext50")

In [ ]:
learn.load("Resnext50")

## Overfitting --> Increasing dropout 

In [ ]:
ps=[0.4, 0.5, 0.9]

In [ ]:
learn = ConvLearner.pretrained(arch, data, ps=ps)
learn.fit(1e-3, 3)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-3, 2, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()

In [ ]:
lr=np.array([1e-6,1e-4,1e-3])

In [ ]:
learn.unfreeze()
learn.fit(lr, 3, cycle_len=2, cycle_mult=3)

### Analyzing Results

There is something else we can do with data augmentation: use it at *inference* time (also known as *test* time). Not surprisingly, this is known as *test time augmentation*, or just *TTA*.

TTA simply makes predictions not just on the images in your validation set, but also makes predictions on a number of randomly augmented versions of them too (by default, it uses the original image along with 4 randomly augmented versions). It then takes the average prediction from these images, and uses that. To use TTA on the validation set, we can use the learner's `TTA()` method.

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)

# Analyzing the results - vizualising the Images

In [ ]:
data.val_y

In [ ]:
# from here we know that 'female_face' is label 0 and 'male_face' is label 1.
data.classes

In [ ]:
# this gives prediction for validation set. Predictions are in log scale
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(dog)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask((preds == data.val_y)==is_correct & (data.val_y == y), mult)

In [ ]:
# 1. A few correct labels at random
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
# 2. A few incorrect labels at random
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [ ]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(most_by_correct(0, True), "Most correct female faces")

In [ ]:
plot_val_with_title(most_by_correct(1, True), "Most correct male faces")

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect female faces")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect male faces")

In [ ]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Most uncertain predictions")

# Conclusion

## When increasing size of image-set the model should improve, but because of the decrease in image-quality and their relevance (as vizualised) it decreases from approximately 90% to 80% accuracy. 

## Though, interesting to see if a more powerful model will have an impact

# Trying with a different model: *Resnet101*

In [ ]:
arch = resnet34
sz = 299 # One of two standards for ImageNet (224)
bs = 32 # 64 resulted in cuda-out-of-memory-error

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, ps=0.2)

In [ ]:
learn.fit(1e-3, 3)

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-4, 2, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save("resnet34")

In [ ]:
lr = [1e-5, 1e-4, 1e-3]

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)

# Conclusion

> ## Improving, but still overfitting 
> ### Though, cannot increase the dropout? 

# Prediction on new data
## From Kinect V2 camera

In [ ]:
print(arch)

In [ ]:
data.classes

In [ ]:
log_preds = learn.predict()
log_preds.shape

In [ ]:
log_preds[:10]

In [ ]:
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])  

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])
        
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [ ]:
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [ ]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [ ]:
plot_val_with_title(most_by_correct(0, True), "Most correct !Person")

In [ ]:
plot_val_with_title(most_by_correct(1, True), "Most correct Person")

In [ ]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect !Person")

In [ ]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect Person")

In [ ]:
trn_tfms, val_tfms = tfms_from_model(arch, sz)

In [ ]:
def from_np_arr_to_jpg(image, data):
    image_load = np.load(image)
    arr2im = Image.fromarray(image_load)
    new_img_path = image[:-4] + ".jpg"
    arr2im.save(new_img_path)

    predict_single_picture_jpg(new_img_path, data)

In [ ]:
def predict_single_picture_jpg(image, data):
    image_load = open_image(image)
    im = val_tfms(np.array(image_load))
    print(im.shape)
    learn.precompute=False
    preds_log = learn.predict_array(im[None])
    preds = np.exp(preds_log)
    predict = np.argmax(preds_log)
    classes = data.classes
    print(classes[predict])

In [ ]:
learn.load("/home/ros/catkin_ws/src/Various_DL/Personal_Projects/Boy_Girl_Classifying/data/models/resnet34")

In [ ]:
image_path = "/home/ros/catkin_ws/src/reading_images/images/"

In [ ]:
image_names = os.listdir(image_path)

In [ ]:
len(image_names)

In [ ]:
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 5
counter = 1
for image in image_names:
    img_p = image_path + image
    fig.add_subplot(rows, columns, counter)
    plt.imshow(np.load(img_p))
    predict_single_picture_np_arr(img_p, data)
    counter += 1
plt.show()

In [ ]:
t_p = ("/home/ros/catkin_ws/src/Various_DL/Personal_Projects/Boy_Girl_Classifying/data/train")

In [ ]:
os.listdir(t_p)

In [ ]:
t_p_person = t_p + "/Person/"

In [ ]:
f_faces = os.listdir(t_p_person)[:30]

In [ ]:
fig = plt.figure(figsize=(8, 8))
columns = 6
rows = 6
counter = 1
for image in f_faces:
    img_p = t_p_person + image
    fig.add_subplot(rows, columns, counter)
    plt.imshow(Image.open(img_p))
    predict_single_picture_jpg(img_p, data)
    counter += 1
plt.show()